In [2]:
import pandas as pd

df_siniestros = pd.read_excel('../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx')
excel_path = '../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx'

In [3]:
###
# 1. Ver todas las hojas
###
print('Hojas disponibles en el archivo:')
hojas = pd.ExcelFile(excel_path).sheet_names
for i, hoja in enumerate(hojas):
    print(f"   {i+1}. {hoja}")

Hojas disponibles en el archivo:
   1. Siniestros
   2. Vehiculos
   3. Actor_vial
   4. Limpieza de datos
   5. Diccionario


In [ ]:
###
# 2. Filtrar hojas que no queremos
###
hojas_interes = []
for hoja in hojas:
    if 'siniestros' in hoja.lower() or 'limpieza' in hoja.lower() or 'diccionario' in hoja.lower():
        continue
    else:
        hojas_interes.append(hoja)

print(f'\nHojas a analizar: {hojas_interes}')


Hojas a analizar: ['Vehiculos', 'Actor_vial']


In [5]:
###
# 3. Ver todas las columnas de cada hoja de interes (función)
###
def analizar_hoja_detallada(hoja_nombre):
    '''Función para análisis detallado de cada hoja'''
    separador = '=' * 60
    print(f'\n{separador}')
    print(f'Analisis detallado - hoja: {hoja_nombre}')
    print(f'{separador}')
    
    # Cargar la hoja
    df_sinistros_clean = pd.read_excel(excel_path, sheet_name=hoja_nombre)
    
    # Información básica
    print(f'Dimensiones: {df_sinistros_clean.shape[0]} filas × {df_sinistros_clean.shape[1]} columnas')
    
    # Columnas
    print(f'\nLista de Columnas ({len(df_sinistros_clean.columns)} total):')
    for i, columna in enumerate(df_sinistros_clean.columns, 1):
        print(f'   {i:2d}. {columna}')
    
    # Tipos de datos
    print(f'\nTipo de dátos:')
    print(df_sinistros_clean.dtypes)
    
    # Valores nulos
    print(f'\nValores nulos (primeras 10 columnas):')
    nulos = df_sinistros_clean.isnull().sum()
    print(nulos)
    
    return df_sinistros_clean



In [6]:
###
# 3. Ver todas las columnas de cada hoja de interes (ejecución)
###
dataframes = {}
for hoja in hojas_interes:
    dataframes[hoja] = analizar_hoja_detallada(hoja)


Analisis detallado - hoja: Vehiculos
Dimensiones: 523168 filas × 32 columnas

Lista de Columnas (32 total):
    1. Codigo_Accidente
    2. Formulario
    3. Fecha_Acc
    4. AA_Acc
    5. Codigo_Vehiculo
    6. Clase
    7. Servicio
    8. Modalidad
    9. Vehiculo_Viajaba_Clasificado
   10. Tipo_SITP
   11. Con_Bicicleta
   12. Con_Carga
   13. Con_Embriaguez
   14. Con_Huecos
   15. Con_Menores
   16. Con_Moto
   17. Con_Peaton
   18. Con_Persona_Mayor
   19. Con_Rutas
   20. Con_Tpi
   21. Con_Tpp
   22. Con_Velocidad
   23. Con_Sitp
   24. Con_Troncal
   25. Con_Alimentador
   26. Con_Zonal
   27. Con_Provisional
   28. Con_Articulado
   29. Con_Biarticulado
   30. Con_Padron_Dual
   31. Con_Servicio_Especial
   32. Con_Taxi

Tipo de dátos:
Codigo_Accidente                         int64
Formulario                              object
Fecha_Acc                       datetime64[ns]
AA_Acc                                   int64
Codigo_Vehiculo                        float64
Clase    

In [7]:
###
# 4. Tabla de analisis para valores unicos vs nulos (función)
###
def generar_tabla_analisis(hoja_nombre):
    """Genera solo la tabla de análisis sin resumen"""
    df = dataframes[hoja_nombre]
    
    separador = '=' * 80
    print(f'\n{separador}')
    print(f'Tabla de Análisis - Hoja: {hoja_nombre}')
    print(f'{separador}')
    
    resultados = []
    
    for columna in df.columns:
        unique_vals = df[columna].nunique()
        total_vals = df.shape[0]
        nulos = df[columna].isnull().sum()
        
        pct_nulos = round((nulos / total_vals * 100), 2) if total_vals > 0 else 0
        pct_unicos = round((unique_vals / total_vals * 100), 2) if total_vals > 0 else 0
        tipo_dato = df[columna].dtype
        
        # Evaluar utilidad
        if pct_nulos > 80:
            utilidad = 'ELIMINAR'
        elif unique_vals == 1:
            utilidad = 'ELIMINAR'
        elif pct_unicos > 90:
            utilidad = 'REVISAR'
        else:
            utilidad = 'MANTENER'
        
        resultados.append({
            'Columna': columna,
            'Tipo': tipo_dato,
            'Unicos': unique_vals,
            '% Unicos': pct_unicos,
            'Nulos': nulos,
            '% Nulos': pct_nulos,
            'Utilidad': utilidad
        })
    
    df_resultados = pd.DataFrame(resultados)
    print(df_resultados.to_string(index=False))
    
    return df_resultados



In [8]:
###
# 4. Tabla de analisis para valores unicos vs nulos (ejecución)
###
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      

In [9]:
###
# 5. Filtrado de eliminar y revisar (ejecución)
###
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)
    
    # Mostrar solo las problemáticas
    problematicas = tabla_resultados[tabla_resultados['Utilidad'] != 'MANTENER']
    if not problematicas.empty:
        print(f'\n--- Columnas problemáticas en {hoja} ---')
        print(problematicas.to_string(index=False))


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      

In [10]:
###
# 6. Eliminar columnas sin información relevante (Función)
###
def eliminar_columnas_vehiculos():
    """Elimina columnas no necesarias de la hoja Vehiculos, excepto excepciones"""
    hoja = 'Vehiculos'
    df_vehiculos = dataframes[hoja].copy()
    
    print(f'=== Eliminación de columnas - {hoja} ===')
    print(f'Dimensiones antes: {df_vehiculos.shape}')
    
    # Primero generar el análisis para ver qué columnas eliminar
    tabla_analisis = generar_tabla_analisis(hoja)
    
    # Identificar columnas a eliminar (excepto excepciones)
    columnas_eliminar = tabla_analisis[
        (tabla_analisis['Utilidad'] == 'ELIMINAR') & 
        (~tabla_analisis['Columna'].isin(['Modalidad', 'Tipo_SITP']))
    ]['Columna'].tolist()
    
    # Columnas que se mantienen por excepción
    columnas_excepciones = tabla_analisis[
        (tabla_analisis['Utilidad'] == 'ELIMINAR') & 
        (tabla_analisis['Columna'].isin(['Modalidad', 'Tipo_SITP']))
    ]['Columna'].tolist()
    
    print(f'\n--- Columnas a ELIMINAR ({len(columnas_eliminar)}) ---')
    for col in columnas_eliminar:
        print(f'   - {col}')
    
    # Eliminar columnas
    df_vehiculos_limpio = df_vehiculos.drop(columns=columnas_eliminar)
    
    print(f'\n--- Resultado ---')
    print(f'Dimensiones después: {df_vehiculos_limpio.shape}')
    print(f'Columnas eliminadas: {len(columnas_eliminar)}')
    print(f'Columnas restantes: {df_vehiculos_limpio.shape[1]}')
    
    # Actualizar el dataframe en el diccionario
    dataframes[hoja] = df_vehiculos_limpio
    
    return df_vehiculos_limpio

In [11]:
###
# 6. Eliminar columnas sin información relevante (Ejecución)
###
df_vehiculos_limpiado = eliminar_columnas_vehiculos()

=== Eliminación de columnas - Vehiculos ===
Dimensiones antes: (523168, 32)

Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                 

In [12]:
###
# 7. Verificar que se mantuvieron las excepciones
###
print('=== VERIFICACIÓN DE EXCEPCIONES ===')
print('Columnas en Vehiculos después de limpieza:')
for col in dataframes['Vehiculos'].columns:
    if col in ['Modalidad', 'Tipo_SITP']:
        print(f'   - {col} - MANTENIDA')
    else:
        print(f'   - {col}')

=== VERIFICACIÓN DE EXCEPCIONES ===
Columnas en Vehiculos después de limpieza:
   - Codigo_Accidente
   - Formulario
   - Fecha_Acc
   - AA_Acc
   - Codigo_Vehiculo
   - Clase
   - Servicio
   - Modalidad - MANTENIDA
   - Vehiculo_Viajaba_Clasificado
   - Tipo_SITP - MANTENIDA


In [13]:
###
# 8. Eliminación de columnas repetidas en otras hojas["Formulario", "Fecha_Acc", "AA_Acc" y "Codigo_Vehiculo"] (Función) 
###
def eliminar_columnas_especificas_vehiculos():
    """Elimina columnas específicas adicionales de Vehiculos"""
    hoja = 'Vehiculos'
    df_vehiculos = dataframes[hoja].copy()
    
    print(f'=== ELliminacion de columnas específicas - {hoja} ===')
    print(f'Dimensiones antes: {df_vehiculos.shape}')
    
    # Columnas a eliminar
    columnas_eliminar = ["Formulario", "Fecha_Acc", "AA_Acc", "Codigo_Vehiculo"]
    
    # Verificar qué columnas existen realmente en el DataFrame
    columnas_existentes = [col for col in columnas_eliminar if col in df_vehiculos.columns]
    columnas_no_encontradas = [col for col in columnas_eliminar if col not in df_vehiculos.columns]
    
    print(f'\n--- Columnas a eliminar ---')
    for col in columnas_eliminar:
        if col in df_vehiculos.columns:
            print(f'    {col} - EXISTE')
        else:
            print(f'    {col} - NO ENCONTRADA')
    
    if columnas_no_encontradas:
        print(f'\n--- Columnas no encontradas (ignoradas) ---')
        for col in columnas_no_encontradas:
            print(f'   - {col}')
    
    if not columnas_existentes:
        print('\n  No hay columnas existentes para eliminar')
        return df_vehiculos
    
    # Eliminar columnas existentes
    df_vehiculos_limpio = df_vehiculos.drop(columns=columnas_existentes)
    
    print(f'\n--- Resultado ---')
    print(f'Dimensiones después: {df_vehiculos_limpio.shape}')
    print(f'Columnas eliminadas: {len(columnas_existentes)}')
    print(f'Columnas restantes: {df_vehiculos_limpio.shape[1]}')
    
    # Mostrar nuevas columnas
    print(f'\n--- Columnas finales en {hoja} ---')
    for i, columna in enumerate(df_vehiculos_limpio.columns, 1):
        print(f'   {i}. {columna}')
    
    # Verificar que las excepciones se mantienen
    excepciones = ['Modalidad', 'Tipo_SITP']
    print(f'\n--- Verificación de excepciones ---')
    for excepcion in excepciones:
        if excepcion in df_vehiculos_limpio.columns:
            print(f'   {excepcion} - MANTENIDA')
        else:
            print(f'   {excepcion} - ELIMINADA (ERROR)')
    
    # Actualizar el dataframe en el diccionario
    dataframes[hoja] = df_vehiculos_limpio
    
    return df_vehiculos_limpio


In [14]:
###
# 8. Eliminación de columnas repetidas en otras hojas (Ejecución)
###
df_vehiculos_final = eliminar_columnas_especificas_vehiculos()

=== ELliminacion de columnas específicas - Vehiculos ===
Dimensiones antes: (523168, 10)

--- Columnas a eliminar ---
    Formulario - EXISTE
    Fecha_Acc - EXISTE
    AA_Acc - EXISTE
    Codigo_Vehiculo - EXISTE

--- Resultado ---
Dimensiones después: (523168, 6)
Columnas eliminadas: 4
Columnas restantes: 6

--- Columnas finales en Vehiculos ---
   1. Codigo_Accidente
   2. Clase
   3. Servicio
   4. Modalidad
   5. Vehiculo_Viajaba_Clasificado
   6. Tipo_SITP

--- Verificación de excepciones ---
   Modalidad - MANTENIDA
   Tipo_SITP - MANTENIDA


In [15]:
###
# 9. Verificar el estado final de Vehiculos
###
print('=== Estado Final - Hoja Vehiculos ===')
print(f'Dimensiones: {dataframes["Vehiculos"].shape}')
print(f'Columnas finales:')

for i, columna in enumerate(dataframes['Vehiculos'].columns, 1):
    # Verificar si son columnas importantes o excepciones
    if columna in ['Modalidad', 'Tipo_SITP']:
        print(f'   {i}. {columna} ← EXCEPCIÓN MANTENIDA')
    else:
        print(f'   {i}. {columna}')

print(f'\nTotal columnas: {len(dataframes["Vehiculos"].columns)}')

=== Estado Final - Hoja Vehiculos ===
Dimensiones: (523168, 6)
Columnas finales:
   1. Codigo_Accidente
   2. Clase
   3. Servicio
   4. Modalidad ← EXCEPCIÓN MANTENIDA
   5. Vehiculo_Viajaba_Clasificado
   6. Tipo_SITP ← EXCEPCIÓN MANTENIDA

Total columnas: 6


In [16]:
###
# 10. Eliminar columnas sin información relevante de Actor_vial (Función)
###
def eliminar_columnas_actor_vial():
    """Elimina columnas no necesarias de la hoja Actor_vial"""
    hoja = 'Actor_vial'
    df_actor_vial = dataframes[hoja].copy()
    
    print(f'=== Eliminación de columnas - {hoja} ===')
    print(f'Dimensiones antes: {df_actor_vial.shape}')
    
    # Primero generar el análisis para ver qué columnas eliminar
    tabla_analisis = generar_tabla_analisis(hoja)
    
    # Identificar columnas a eliminar (solo las marcadas como ELIMINAR)
    columnas_eliminar = tabla_analisis[
        tabla_analisis['Utilidad'] == 'ELIMINAR'
    ]['Columna'].tolist()
    
    print(f'\n--- Columnas a ELIMINAR ({len(columnas_eliminar)}) ---')
    for col in columnas_eliminar:
        print(f'   - {col}')
    
    # Eliminar columnas
    df_actor_vial_limpio = df_actor_vial.drop(columns=columnas_eliminar)
    
    print(f'\n--- Resultado ---')
    print(f'Dimensiones después: {df_actor_vial_limpio.shape}')
    print(f'Columnas eliminadas: {len(columnas_eliminar)}')
    print(f'Columnas restantes: {df_actor_vial_limpio.shape[1]}')
    
    # Actualizar el dataframe en el diccionario
    dataframes[hoja] = df_actor_vial_limpio
    
    return df_actor_vial_limpio

In [17]:
###
# 10. Eliminar columnas sin información relevante de Actor_vial (Ejecución)
###
df_actor_vial_limpiado = eliminar_columnas_actor_vial()

=== Eliminación de columnas - Actor_vial ===
Dimensiones antes: (608155, 45)

Tabla de Análisis - Hoja: Actor_vial
                       Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidentado          int64  608155    100.00      0     0.00  REVISAR
              Codigo_Accidente          int64  278463     45.79      0     0.00 MANTENER
                    Formulario         object  278463     45.79      0     0.00 MANTENER
               Codigo_Vehiculo        float64      31      0.01  12686     2.09 MANTENER
               Ccodigo_Victima          int64      72      0.01      0     0.00 MANTENER
                      FechaAcc datetime64[ns]    3653      0.60      0     0.00 MANTENER
                       AnnoAcc          int64      10      0.00      0     0.00 MANTENER
                        mesAcc         object      12      0.00      0     0.00 MANTENER
                    DD_Mes_Acc          int64      31      0.01      0     0.00 MANT

In [18]:
###
# 11. Verificar el estado final de Actor_vial
###
print('=== ESTADO FINAL - HOJA ACTOR_VIAL ===')
print(f'Dimensiones: {dataframes["Actor_vial"].shape}')
print(f'Columnas finales:')

for i, columna in enumerate(dataframes['Actor_vial'].columns, 1):
    print(f'   {i}. {columna}')

print(f'\nTotal columnas: {len(dataframes["Actor_vial"].columns)}')

=== ESTADO FINAL - HOJA ACTOR_VIAL ===
Dimensiones: (608155, 21)
Columnas finales:
   1. Codigo_Accidentado
   2. Codigo_Accidente
   3. Formulario
   4. Codigo_Vehiculo
   5. Ccodigo_Victima
   6. FechaAcc
   7. AnnoAcc
   8. mesAcc
   9. DD_Mes_Acc
   10. Dia_Semana_Acc
   11. hourAcc
   12. minuAcc
   13. Localidad
   14. Edad
   15. Sexo
   16. Gravedad_Indicador_Tradicional
   17. Muerte_Posterior
   18. Gravedad_Indicador_30d
   19. Condicion
   20. Condicion_Especifica
   21. Validación

Total columnas: 21


In [19]:
###
# 12. Eliminar columnas específicas de Actor_vial ["Formulario", "Codigo_Vehiculo", "Ccodigo_Victima", "FechaAcc", "AnnoAcc", "mesAcc", "DD_Mes_Acc", "Dia_Semana_Acc", "hourAcc", "minuAcc", "Localidad"] (Función)
###
def eliminar_columnas_especificas_actor_vial():
    """Elimina columnas específicas adicionales de Actor_vial"""
    hoja = 'Actor_vial'
    df_actor_vial = dataframes[hoja].copy()
    
    print(f'=== Eliminación de columnas específicas - {hoja} ===')
    print(f'Dimensiones antes: {df_actor_vial.shape}')
    
    # Columnas a eliminar
    columnas_eliminar = ["Formulario", "Codigo_Vehiculo", "Ccodigo_Victima", "FechaAcc", 
                        "AnnoAcc", "mesAcc", "DD_Mes_Acc", "Dia_Semana_Acc", "hourAcc", 
                        "minuAcc", "Localidad"]
    
    # Verificar qué columnas existen realmente en el DataFrame
    columnas_existentes = [col for col in columnas_eliminar if col in df_actor_vial.columns]
    columnas_no_encontradas = [col for col in columnas_eliminar if col not in df_actor_vial.columns]
    
    print(f'\n--- Columnas a eliminar ---')
    for col in columnas_eliminar:
        if col in df_actor_vial.columns:
            print(f'   - {col} - EXISTE')
        else:
            print(f'   - {col} - NO ENCONTRADA')
    
    if columnas_no_encontradas:
        print(f'\n--- Columnas no encontradas (ignoradas) ---')
        for col in columnas_no_encontradas:
            print(f'   - {col}')
    
    if not columnas_existentes:
        print('\nNo hay columnas existentes para eliminar')
        return df_actor_vial
    
    # Eliminar columnas existentes
    df_actor_vial_limpio = df_actor_vial.drop(columns=columnas_existentes)
    
    print(f'\n--- Resultado ---')
    print(f'Dimensiones después: {df_actor_vial_limpio.shape}')
    print(f'Columnas eliminadas: {len(columnas_existentes)}')
    print(f'Columnas restantes: {df_actor_vial_limpio.shape[1]}')
    
    # Mostrar nuevas columnas
    print(f'\n--- Columnas finales en {hoja} ---')
    for i, columna in enumerate(df_actor_vial_limpio.columns, 1):
        print(f'   {i}. {columna}')
    
    # Actualizar el dataframe en el diccionario
    dataframes[hoja] = df_actor_vial_limpio
    
    return df_actor_vial_limpio

In [20]:
###
# 12. Eliminación de columnas repetidas en otras hojas (Ejecución)
###
df_actor_vial_final = eliminar_columnas_especificas_actor_vial()

=== Eliminación de columnas específicas - Actor_vial ===
Dimensiones antes: (608155, 21)

--- Columnas a eliminar ---
   - Formulario - EXISTE
   - Codigo_Vehiculo - EXISTE
   - Ccodigo_Victima - EXISTE
   - FechaAcc - EXISTE
   - AnnoAcc - EXISTE
   - mesAcc - EXISTE
   - DD_Mes_Acc - EXISTE
   - Dia_Semana_Acc - EXISTE
   - hourAcc - EXISTE
   - minuAcc - EXISTE
   - Localidad - EXISTE

--- Resultado ---
Dimensiones después: (608155, 10)
Columnas eliminadas: 11
Columnas restantes: 10

--- Columnas finales en Actor_vial ---
   1. Codigo_Accidentado
   2. Codigo_Accidente
   3. Edad
   4. Sexo
   5. Gravedad_Indicador_Tradicional
   6. Muerte_Posterior
   7. Gravedad_Indicador_30d
   8. Condicion
   9. Condicion_Especifica
   10. Validación


In [ ]:
###
# 13. Mostrar columnas finales de cada hoja procesada
###
print("=== Columnas finales - Hoja de Vehiculos ===")
print(f"Total columnas: {len(df_vehiculos_final.columns)}")
print("Lista de columnas:")
for i, columna in enumerate(df_vehiculos_final.columns, 1):
    print(f"  {i}. {columna}")

print("\n" + "="*50)

print("\n=== Columnas finales - Hoja de Actor_vial ===")
print(f"Total columnas: {len(df_actor_vial_final.columns)}")
print("Lista de columnas:")
for i, columna in enumerate(df_actor_vial_final.columns, 1):
    print(f"  {i}. {columna}")

=== Columnas finales - Hoja de Vehiculos ===
Total columnas: 6
Lista de columnas:
  1. Codigo_Accidente
  2. Clase
  3. Servicio
  4. Modalidad
  5. Vehiculo_Viajaba_Clasificado
  6. Tipo_SITP


=== Columnas finales - Hoja de Actor_vial ===
Total columnas: 10
Lista de columnas:
  1. Codigo_Accidentado
  2. Codigo_Accidente
  3. Edad
  4. Sexo
  5. Gravedad_Indicador_Tradicional
  6. Muerte_Posterior
  7. Gravedad_Indicador_30d
  8. Condicion
  9. Condicion_Especifica
  10. Validación


In [ ]:
###
# 14. Crear nuevo documento SOLO con las 2 hojas limpias
###
ruta_nuevo_excel = "../data/processed/df_siniestros_limpio.xlsx"

with pd.ExcelWriter(ruta_nuevo_excel, engine='openpyxl') as writer:
    df_vehiculos_final.to_excel(writer, sheet_name='Vehiculos', index=False)
    df_actor_vial_final.to_excel(writer, sheet_name='Actor_Vial', index=False)

print(f"CREADO: {ruta_nuevo_excel}")
print("Contenido: Actor_Vial, Vehiculos (ambas limpias)")

CREADO: ../data/processed/base_anuario_de_siniestralidad_clean.xlsx
Contenido: Actor_Vial, Vehiculos (ambas limpias)
